In [50]:
import os, sys
from pyspark import SparkContext, SparkConf
appName = "miApp"
master="local[*]"

conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

In [51]:
#print(os.getcwd())
textFile = sc.textFile("../../datasets/worldbank-millennium-development-goals.csv")

# datasets at:
#https://datasource.kapsarc.org/explore/dataset/worldbank-millennium-development-goals/export/
#https://datasource.kapsarc.org/explore/dataset/worldbank-health-nutrition-and-population-statistics/export/

In [52]:
print(textFile.count()) # Number of items in this RDD

textFile.first()  # First item in this RDD

98939


'Country Name;Country Code;Indicator Name;Indicator Code;Year;Value'

In [41]:
textFile.filter(lambda line: "Spain" in line).count()  # How many lines contain "Spain"?

144

In [42]:
# find the line with the most words (lambda format)
textFile.map(lambda line: len(line.split())).reduce(lambda a, b: a if (a > b) else b)

26

In [43]:
# find the line with the most words (function format)
def max(a, b):
     if a > b:
         return a
     else:
         return b


textFile.map(lambda line: len(line.split())).reduce(max)

26

In [53]:
# One common data flow pattern is MapReduce, as popularized by Hadoop. Spark can implement MapReduce flows easily:
wordCounts = textFile.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [54]:
wordCounts.collect()

[('Country', 1),
 ('Code;Indicator', 1),
 ('Code;Year;Value', 1),
 ('cases', 761),
 ('reported;SH.STA.MALR;1998;', 12),
 ('reported;SH.STA.MALR;2000;', 25),
 ('Grenada;GRD;Marine', 6),
 ('protected', 1432),
 ('areas', 1432),
 ('of', 62431),
 ('waters);ER.MRN.PTMR.ZS;1993;', 8),
 ('waters);ER.MRN.PTMR.ZS;1995;', 13),
 ('waters);ER.MRN.PTMR.ZS;2003;0.022562666963014', 1),
 ('ratio', 4522),
 ('estimate,', 766),
 ('live', 2259),
 ('births);SH.STA.MMRT;2013;27.0', 1),
 ('births);SH.STA.MMRT;1992;38.0', 1),
 ('Grenada;GRD;Mortality', 7),
 ('rate,', 6656),
 ('(per', 5200),
 ('Grenada;GRD;Net', 39),
 ('developed', 9691),
 ('countries', 13184),
 ('US$);DC.ODA.TLDC.CD;1994;', 25),
 ('US$);DC.ODA.TLDC.CD;2001;', 23),
 ('GNI);DC.ODA.TOTL.GN.ZS;2009;', 27),
 ('(constant', 3005),
 ('2012', 779),
 ('US$);DC.ODA.TOTL.KD;2010;', 23),
 ('US$);DC.ODA.TOTL.KD;1990;', 30),
 ('US$);DC.ODA.TOTL.KD;1998;', 28),
 ('GNI);DT.ODA.ODAT.GN.ZS;2011;1.63713435795736', 1),
 ('GNI);DT.ODA.ODAT.GN.ZS;2001;2.294364539132

In [103]:
#textFile.first() # returns first item
textFile.take(3)

['Country Name;Country Code;Indicator Name;Indicator Code;Year;Value',
 'Grenada;GRD;Malaria cases reported;SH.STA.MALR;1998;',
 'Grenada;GRD;Malaria cases reported;SH.STA.MALR;2000;']

In [105]:
lines = textFile.map(lambda x: (x.split(',')))

In [107]:
lines.take(3)

[['Country Name;Country Code;Indicator Name;Indicator Code;Year;Value'],
 ['Grenada;GRD;Malaria cases reported;SH.STA.MALR;1998;'],
 ['Grenada;GRD;Malaria cases reported;SH.STA.MALR;2000;']]

In [116]:
#get Year
years = lines.map(lambda x: (x[0].split(';')[4], 1))
print(years.take(3))

counts = years.reduceByKey(lambda a, b: a + b)
print(counts.collect()) # TODO: fix it

[('Year', 1), ('1998', 1), ('2000', 1)]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 68.0 failed 1 times, most recent failure: Lost task 0.0 in stage 68.0 (TID 111, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\spark\python\pyspark\rdd.py", line 2408, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\spark\python\pyspark\rdd.py", line 2408, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\spark\python\pyspark\rdd.py", line 345, in func
    return f(iterator)
  File "C:\spark\python\pyspark\rdd.py", line 1827, in combineLocally
    merger.mergeValues(iterator)
  File "C:\spark\python\lib\pyspark.zi

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 42.0 failed 1 times, most recent failure: Lost task 1.0 in stage 42.0 (TID 82, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-92-11ed8edc80e6>", line 2, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonR

In [ ]:
#EXAMPLE
pairs = people.map(lambda x: (x[4], 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
print(counts.collect())